# Package and drive connect


In [ ]:
!pip -qq install ninja

Connect google drive with cloud

In [ ]:
import os.path as path 
if not path.exists("/content/drive"):
  !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !sudo apt-get update -qq 2>&1 > /dev/null
  !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
  !google-drive-ocamlfuse

  !sudo apt-get install -qq w3m # to act as web browser 
  !xdg-settings set default-web-browser w3m.desktop # to set default browser
  %cd /content
  !mkdir drive
  %cd drive
  !mkdir MyDrive
  %cd ..
  %cd ..
  !google-drive-ocamlfuse /content/drive/MyDrive

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os.path
import io
import IPython.display
import numpy as np
import cv2
import PIL.Image
import PIL
import ipywidgets as widgets
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dir = '/content/drive/MyDrive/11785Project/Model'
os.chdir(dir)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

# Step 1: Data Preparation 

1. dataset in ./training_runs/dataset, select 5%-10% data from HHFQ 1024 x 1024

2. create files to ./training_runs/dataset/{args.dataset_name}

        wp.npy, w.npy, z.npy, hair_scores.npy and gender_scores.npy
            hair_scores, 1: not bald, 0: bald 
            gender_score, 1: male, 0: female 
            D0: without noise
            Dnoise: add noise

In [ ]:
class args:
  def __init__(self):
    self.dataset_name = 'D0'
    self.num = 2500 # Number of images to generate, default = 20000
    self.truncation_psi = 0.8
    self.bald_only = False
    self.add_noise = False
    self.save_StyleSpace = False
    self.model_name = 'stylegan2_ada'
arg = args()

In [ ]:
from generate_data import run

In [ ]:
run(arg)

In [ ]:
class args:
  def __init__(self):
    self.dataset_name = 'Dnoise'
    self.num = 2500 # Number of images to generate, default = 20000
    self.truncation_psi = 0.8
    self.bald_only = False
    self.add_noise = True
    self.save_StyleSpace = False
    self.model_name = 'stylegan2_ada'
arg = args()

In [ ]:
run(arg)

# Step 2: Boundary Training 

## Train boundary (Interface gan++)

In [ ]:
class args:
  def __init__(self):
    self.output_dir = './data/boundaries/test/coarse/stylegan2_ffhq_hair_w_male'
    self.dataset_path = './training_runs/dataset/D0'
    self.split_ratio = 0.98
    self.invalid_value = None
args = args()

In [ ]:
from train_man_hair_coarse_boundary import run

In [ ]:
run(args)

## Pretrained boundary from interface gan

./data/boundaries/stylegan2_ada/coarse/stylegan2_ffhq_hair_w_male

./data/boundaries/stylegan2_ada/coarse/stylegan2_ffhq_gender_styleflow

# Step 3: Male Hair Removal

Training bald male data using hair boundary, latent_space_type = 'wp'

create files in './training_runs/male_training' 

    temp_codes,
    mask: save hair_mask (including hat)
    temp_imgs, 
    res_wp_codes, 
    res_img: viz_result(image after synthesis and diffusion)


In [ ]:
class args():
  def __init__(self):
    self.dataset_name = 'D0'
    self.num = 1000 # Training data num, 2000
    self.hair_boundary_dir = './data/boundaries/stylegan2_ada/coarse/stylegan2_ffhq_hair_w_male'
    self.truncation_psi =0.75
    self.save_temp = False
    self.learning_rate = 0.01
    self.num_iterations = 150 
    self.loss_weight_feat = 5e-5 # The perceptual loss weigh
    self.loss_weight_id = 1.0 # The facial identity loss weight
args = args()

In [ ]:
from train_bald_male_data import run

In [ ]:
run(args)

In [ ]:
class args():
  def __init__(self):
    self.dataset_name = 'Dnoise'
    self.num = 1000 # Training data num, 2000
    self.hair_boundary_dir = './data/boundaries/stylegan2_ada/coarse/stylegan2_ffhq_hair_w_male'
    self.truncation_psi =0.75
    self.save_temp = False
    self.learning_rate = 0.01
    self.num_iterations = 150
    self.loss_weight_feat = 5e-5 # The perceptual loss weigh
    self.loss_weight_id = 1.0 # The facial identity loss weight
args = args()

In [ ]:
run(args)

# Step 4: Male Mapper Training

Edit image synthesis with given semantic boundary

1. Create files to ./training_runs/{args.mapper_name}/data

        train.txt
        val.txt
        test.txt

2. Train Hairmapper for man


In [ ]:
class args():
  def __init__(self):
    self.dataset_name = 'D0'
    self.noise_dataset_name = 'Dnoise'
    self.test_data_dir = './data/test_data/man'
    self.mapper_name = 'male_mapper_large'
args = args()

In [ ]:
from male_mapper_data_preparation import run

In [ ]:
run(args)

In [ ]:
class args():
  def __init__(self):
    self.mapper_name = 'male_mapper_large'
    self.max_steps = 10000 # epochs, default=100000
    self.learning_rate = 0.005
    self.val_interval = 200 # each val_interval do validate
    self.board_interval = 50
    self.image_interval = 100 # each image_interval process image and save
    self.save_interval = 200 # each save_interval save model
    self.alpha = 0.4  # alpha is a hyper-parameter that controls the distance that w+ moves along nh, (3), in original code, there might be a bug in validation *1.2(comment)
    self.input_dim = 512
    # (8) in paper
    self.id_lambda = 0.1  # loss_id weight in loss, L_id
    self.img_l2_lambda_origin = 0.4 # loss_l2_origin_img weight in loss, torch.mean(((origin_img - x_hat) * mask) ** 2, dim=[0, 1, 2, 3]) L_face
    self.img_l2_lambda_res = 0.4  # loss_l2_res_img weight in loss, torch.mean(((res_x - x_hat)) ** 2, dim=[0, 1, 2, 3]) L_hair
    self.latent_l2_lambda = 0.1  # loss_l2_latent weight in loss,  nn.MSELoss(w_hat, res_w), L_latent
    self.truncation_psi =0.8 # MappingNetwork, if truncation_psi !=1, apply truncate x = self.w_avg.lerp(x, truncation_psi)
    self.resume = ''
args = args()

In [ ]:
from train_mapper import Trainer

In [ ]:
trainer = Trainer(args)
trainer.train()

# Step 5: Female Hair Removal

In [ ]:
class args():
  def __init__(self):
    self.dataset_name = 'D0'
    self.num = 1000 
    self.male_mapper_name = 'male_mapper_large'
    self.mapper_ckpt_path = ''
    self.truncation_psi = 0.8
    self.gender_boundary_dir = './data/boundaries/stylegan2_ada/coarse/stylegan2_ffhq_gender_styleflow'
    self.save_temp = False
args = args()

In [ ]:
from train_bald_female_data import run

In [ ]:
run(args)

In [ ]:
class args():
  def __init__(self):
    self.dataset_name = 'Dnoise'
    self.num = 1000
    self.male_mapper_name = 'male_mapper_large'
    self.mapper_ckpt_path = ''
    self.truncation_psi = 0.8
    self.gender_boundary_dir = './data/boundaries/stylegan2_ada/coarse/stylegan2_ffhq_gender_styleflow'
    self.save_temp = False
args = args()

In [ ]:
run(args)

# Step 6: Final Mapper Training

Edit image synthesis with given semantic boundary

save image in log

    train/val: res_x(bald image), x_hat(synthetic image), origin_img
    test: x_hat, res(origin_img * mask + x_hat * (1 - mask)), origin_img

In [ ]:
class args:
  def __init__(self):
    self.test_data_dir = './data/test_data/final'
    self.dataset_name = 'D0'
    self.noise_dataset_name = 'Dnoise'
    self.mapper_name = 'final_mapper_large'
args = args()

In [ ]:
from final_mapper_data_preparation import run

In [ ]:
run(args)

In [ ]:
class args():
  def __init__(self):
    self.mapper_name = 'final_mapper_large'
    self.max_steps = 10000 # epochs, default=100000
    self.learning_rate = 0.005
    self.val_interval = 200 # each val_interval do validate
    self.board_interval = 50
    self.image_interval = 100 # each image_interval process image and save
    self.save_interval = 200 # each save_interval save model
    self.alpha = 0.4  # alpha is a hyper-parameter that controls the distance that w+ moves along nh, (3), in original code, there might be a bug in validation *1.2(comment)
    self.input_dim = 512
    # (8) in paper
    self.id_lambda = 0.1  # loss_id weight in loss, L_id
    self.img_l2_lambda_origin = 0.4 # loss_l2_origin_img weight in loss, torch.mean(((origin_img - x_hat) * mask) ** 2, dim=[0, 1, 2, 3]) L_face
    self.img_l2_lambda_res = 0.4  # loss_l2_res_img weight in loss, torch.mean(((res_x - x_hat)) ** 2, dim=[0, 1, 2, 3]) L_hair
    self.latent_l2_lambda = 0.1  # loss_l2_latent weight in loss,  nn.MSELoss(w_hat, res_w), L_latent
    self.truncation_psi =0.8 # MappingNetwork, if truncation_psi !=1, apply truncate x = self.w_avg.lerp(x, truncation_psi)
    self.resume = ''
args = args()

In [ ]:
from train_mapper import Trainer

In [ ]:
trainer = Trainer(args)
trainer.train()

# Testing

## Image align


In [ ]:
import os
import sys
import bz2
from keras.utils import get_file
from ffhq_dataset.face_alignment import image_align
from ffhq_dataset.landmarks_detector import LandmarksDetector
LANDMARKS_MODEL_URL = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'

def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path

def trans(RAW_IMAGES_DIR, ALIGNED_IMAGES_DIR):
    landmarks_model_path = unpack_bz2(get_file('shape_predictor_68_face_landmarks.dat.bz2',
                                               LANDMARKS_MODEL_URL, cache_subdir='temp'))
    RAW_IMAGES_DIR = RAW_IMAGES_DIR
    ALIGNED_IMAGES_DIR = ALIGNED_IMAGES_DIR

    landmarks_detector = LandmarksDetector(landmarks_model_path)
    for img_name in os.listdir(RAW_IMAGES_DIR):
        raw_img_path = os.path.join(RAW_IMAGES_DIR, img_name)
        for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(raw_img_path), start=1):
            face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], i)
            aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)

            image_align(raw_img_path, aligned_face_path, face_landmarks)

## Test

Put Test image in ./test_data/origin

In [ ]:
os.chdir('./encoder4editing')

In [ ]:
from argparse import Namespace
import sys
import torch
import torchvision.transforms as transforms
import numpy as np
import PIL.Image
from PIL import ImageFile
import glob
import os
import argparse
sys.path.append(".")
sys.path.append("..")
ImageFile.LOAD_TRUNCATED_IMAGES = True
from models.psp import pSp

In [ ]:
class args:
  def __init__(self):
    self.data_dir = '../test_data'
args = args()

In [ ]:
from encoder4editing.encode import run

In [ ]:
run(args)

Load our model

In [ ]:
class args:
  def __init__(self):
    self.data_dir = './test_data'
    #self.model = './training_runs/final_mapper/checkpoints/best_model.pt' #our final model
    self.model = './training_runs/male_mapper_large/checkpoints/best_model.pt' # our male model
    self.learning_rate = 0.01
    self.num_iterations = 100
    self.loss_weight_feat = 3e-5
    self.loss_weight_id = 1.0
    self.remain_ear = False
    self.diffuse = False  #With diffuse provide better results
    self.dilate_kernel_size = 50
    self.blur_kernel_size = 30
    self.truncation_psi =0.75
args = args()

In [ ]:
os.chdir(dir)

In [ ]:
from main_mapper import run

In [ ]:
run(args)

##  visualize results

In [ ]:
from IPython.display import display
import glob
import cv2
import numpy as np
from PIL import Image

In [ ]:
for res_path in glob.glob('./test_data/mapper_res/*'):
    res_img = cv2.imread(res_path)[:,:,::-1]
    input_path = res_path.replace('mapper_res','origin')
    input_img = cv2.imread(input_path)[:,:,::-1]
    visualize = np.concatenate([res_img,input_img], axis=1)
    res_im = Image.fromarray(visualize)
    display(res_im)